In [47]:
import os
import pandas as pd
from datetime import date
import snscrape.modules.twitter as sntwitter
import pandas
from getTopic import getTopic,getTopicv2
from IPython.display import clear_output

def getUrl(source,_from,_to):
    tweets_list2 = []
    query = f'from:{source} since:{_from} until:{_to}'
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
        tweets_list2.append([tweet.id,tweet.user.id,tweet.user.username,tweet.user.displayname,tweet.user.location,tweet.user.description,tweet.user.descriptionUrls,tweet.content,tweet.date.strftime("%Y-%m-%d %H:%M:%S"),tweet.hashtags,tweet.inReplyToTweetId,tweet.inReplyToUser,tweet.user.followersCount,tweet.user.followersCount,tweet.user.created,tweet.lang,tweet.retweetCount,
                             tweet.likeCount,tweet.lang])
    return pd.DataFrame(tweets_list2,columns=['tweetid','userid','user_display_name','user_screen_name','user_reported_location','user_profile_description','user_profile_url','tweet_text','tweet_time','hashtags','in_reply_to_tweetid','in_reply_to_userid','follower_count','following_count','account_creation_date','account_language','retweet_count',
                                              'like_count','tweet_language'])
    # return sntwitter.TwitterSearchScraper(f'from:{source} since:{_from} until:{_to}').get_items()

def getUrl2(source,_from,_to):
    tweets_list2 = []
    query = f'from:{source} since:{_from} until:{_to}'
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
        tweets_list2.append(tweet)
    return tweets_list2
    # return sntwitter.TwitterSearchScraper(f'from:{source} since:{_from} until:{_to}').get_items()

In [48]:
fileName = 'FNsample4'
df = pd.read_csv("../../../data/raw/excel_files/"+fileName+".csv")
df['tweet_time'] = pd.to_datetime(df['tweet_time'])

In [49]:
timeRange = list(df.groupby([df['tweet_time'].dt.date]).groups.keys())
_from = timeRange[0]
_to = timeRange[-1]
print(_from,_to)

2016-09-26 2016-10-02


In [50]:
from getTopic import handleData
from getTopic import df2tweet
from tqdm.auto import tqdm
from datetime import datetime,timedelta
import json

listNews = pd.read_excel("data/News List and dictionary.xlsx",sheet_name='News list')
listNews = listNews[pd.isna(listNews["Type"])]

def getFromsource(_from,_to):
    for i in tqdm(range(0,df_raw.shape[0])):
        try:
            [top_n_topics_df,keywords_appearing_df] = getTopic(None,df2tweet(df_raw.iloc[i]))
            result.append([top_n_topics_df,keywords_appearing_df])
            # if(top_n_topics_df['score1'].values[0] != 0):
            #     print(i, top_n_topics_df['top1'].values, top_n_topics_df['score1'].values[0])
        except:
            result.append([None])
            print('fail')
        # clear_output(wait=True)

In [51]:
# _df = getUrl(listNews.iloc[2]['Twitter Link'].split('/')[-1],list(df.groupby([df['tweet_time'].dt.date]).groups.keys())[0],list(df.groupby([df['tweet_time'].dt.date]).groups.keys())[0]+ timedelta(days=1))
# [top_n_topics_df,keywords_appearing_df] = getTopic(None,df2tweet(_df.iloc[0]))
# _df.iloc[0]

In [52]:
# 492,509, 520, 539, 560,576
result = []
df_raw = pd.read_csv('data/'+fileName+'_real.csv')
getFromsource(_from,_to)
df2 = df_raw.loc[:,['tweetid','userid','user_display_name','tweet_time']]

fail



In [53]:
def addValue(df2,i,keyIndex):
    if((result[i] != None)):
        if(len(result[i])==2):
            c = result[i][0]['top'+str(keyIndex)].values[0]
            df2.at[i,c] = result[i][0]['score'+str(keyIndex)].values[0]
            df2.at[i,c+"_keywords"] = str(result[i][1][c+'_keywords'].values[0])

In [54]:
topics_cols=[]
with open("data/keywords.json", "r") as read_file:
    topics_cols =  list(json.load(read_file))
for c in topics_cols:
    df2[c]=0
    df2[c+"_keywords"]='()'

for i in tqdm(range(0,df_raw.shape[0])):
    addValue(df2,i,1)
    addValue(df2,i,2)
    addValue(df2,i,3)
    addValue(df2,i,4)
    addValue(df2,i,5)
df2.to_csv('data/'+fileName+'_real_topic.csv',index=False)

In [55]:
result[346]

[      top1               top2               top3   top4    top5  score1  \
 0  Traffic  Weather (Regular)  Weather (Extreme)  Crime  Events     0.0   
 
    score2  score3  score4  score5  
 0     0.0     0.0     0.0     0.0  ,
   Traffic_keywords Weather (Regular)_keywords Weather (Extreme)_keywords  \
 0               ()                         ()                         ()   
 
   Crime_keywords Events_keywords Lifestyle_keywords Entertainment_keywords  \
 0             ()              ()                 ()                     ()   
 
   Sports_keywords Education_keywords Politics_keywords  ... Leisure_keywords  \
 0              ()                 ()                ()  ...               ()   
 
   Obituaries_keywords Abortion_keywords 2nd Amendment_keywords  \
 0                  ()                ()                     ()   
 
   Immigration_keywords Police Brutality_keywords Terrorism_keywords  \
 0                   ()                        ()                 ()   
 
   Race_k